In [1]:
import os
import sys
path_to_this_notebook = os.path.abspath('.')
path_to_project = path_to_this_notebook[:path_to_this_notebook.find('note')]
sys.path.append(path_to_project)

from src.planners.socp.socp_planner import SOCPPlanner
from src.save_and_load_tools.save_load_utils import *
from src.simulation_tools.simulate import simulate
from src.grid.grid_env import GridEnv
import time
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import gc

from IPython.display import HTML, Javascript, display

def restart_kernel_and_run_all_cells():
    display(HTML(
        '''
            <script>
                code_show = false;
                function restart_run_all(){
                    IPython.notebook.kernel.restart();
                    setTimeout(function(){
                        IPython.notebook.execute_all_cells();
                    }, 10000)
                }
                function code_toggle() {
                    if (code_show) {
                        $('div.input').hide(200);
                    } else {
                        $('div.input').show(200);
                    }
                    code_show = !code_show
                }
                code_toggle() 
                restart_run_all()
            </script>

        '''
    ))

%matplotlib inline

In [2]:
# Settings

n_scenarios = 999
shuffle = False
overwrite_existing = False


In [3]:
# Planners with different acess to EV locations
mosek_params = {'basisRelTolS': 1e-9,
                'basisTolS': 1e-9, 
                'intpntCoTolDfeas': 1e-9}
obj_factors = [5e5, 1e3, 1e1]

planner_full = SOCPPlanner(observe_ev_locations='full', future_model='known-future', 
                           n_future_samples=5, grid_transformation=None,
                           normalize=True,  obj_factors=obj_factors, tee=False, **mosek_params)
planner_locations_present = SOCPPlanner(observe_ev_locations='present', future_model='known-future', 
                                        n_future_samples=5, grid_transformation=None,
                                        normalize=True,  obj_factors=obj_factors, tee=False, **mosek_params)
planner_locations_past = SOCPPlanner(observe_ev_locations='past', future_model='known-future', 
                                     n_future_samples=5, grid_transformation=None,
                                     normalize=True,  obj_factors=obj_factors, tee=False, **mosek_params)
planner_locations_blind = SOCPPlanner(observe_ev_locations='blind', future_model='known-future', 
                                        n_future_samples=5, grid_transformation=None,
                                        normalize=True,  obj_factors=obj_factors, tee=False, **mosek_params)

# Planners with different grid transformations

planner_single_node = SOCPPlanner(observe_ev_locations='full', future_model='known-future', 
                                  n_future_samples=5, grid_transformation='single-node',
                                  normalize=True,  obj_factors=obj_factors, tee=False, 
                                  use_weird_sur_grid=False, **mosek_params)
planner_parallel = SOCPPlanner(observe_ev_locations='full', future_model='known-future', 
                               n_future_samples=5, grid_transformation='parallel',
                               normalize=True,  obj_factors=obj_factors, tee=False, 
                               use_weird_sur_grid=False, **mosek_params)

planner_single_node_uni = SOCPPlanner(observe_ev_locations='full', future_model='known-future', 
                                  n_future_samples=5, grid_transformation='single-node',
                                  normalize=True,  obj_factors=obj_factors, tee=False, 
                                  use_weird_sur_grid=False, unify_grid=True, **mosek_params)
planner_parallel_uni = SOCPPlanner(observe_ev_locations='full', future_model='known-future', 
                                   n_future_samples=5, grid_transformation='parallel',
                                   normalize=True,  obj_factors=obj_factors, tee=False, 
                                   use_weird_sur_grid=False,unify_grid=True, **mosek_params)

planner_single_node_weird = SOCPPlanner(observe_ev_locations='full', future_model='known-future', 
                                        n_future_samples=5, grid_transformation='single-node',
                                        normalize=True,  obj_factors=obj_factors, tee=False,
                                        use_weird_sur_grid=True, **mosek_params)

planner_parallel_weird = SOCPPlanner(observe_ev_locations='full', future_model='known-future', 
                                     n_future_samples=5, grid_transformation='parallel',
                                     normalize=True,  obj_factors=obj_factors, tee=False,
                                     use_weird_sur_grid=True, **mosek_params)

# Planners with different future models
planner_no_future = SOCPPlanner(observe_ev_locations='full', future_model='no-future', 
                                n_future_samples=5, grid_transformation=None,
                                normalize=True,  obj_factors=obj_factors, tee=False, **mosek_params)
planner_ts_1 = SOCPPlanner(observe_ev_locations='full', future_model='sample', 
                                n_future_samples=1, grid_transformation=None,
                                normalize=True,  obj_factors=obj_factors, tee=False, **mosek_params)
planner_ts_5 = SOCPPlanner(observe_ev_locations='full', future_model='sample', 
                                n_future_samples=5, grid_transformation=None,
                                normalize=True,  obj_factors=obj_factors, tee=False, **mosek_params)


In [4]:
planners = [\
            planner_full,
            #planner_locations_present,
            #planner_locations_past,
            planner_locations_blind,
            planner_single_node, 
            #planner_single_node_uni,
            #planner_single_node_weird,
            planner_parallel,
            #planner_parallel_uni,
            #planner_parallel_weird,
            #planner_no_future,
            #planner_ts_1,
            #planner_ts_5
           ]

for p in planners:
    print(p.name)

SOCPPlanner_normalize=True_future=known-future_ev-locations=full_None
SOCPPlanner_normalize=True_future=known-future_ev-locations=blind_None
SOCPPlanner_normalize=True_future=known-future_ev-locations=full_single-node
SOCPPlanner_normalize=True_future=known-future_ev-locations=full_parallel


In [5]:
experiment_folder = 'IEEE16'
ptu_int = 30

path_to_experiments = path_to_project + '/experiments/%s/' % experiment_folder
print(path_to_experiments)
print(os.listdir(path_to_experiments))


/home/gr1/Projects/EVCP_partially_observable_locations//experiments/IEEE16/
['IEEE16meshed_con']


In [6]:
experiments = os.listdir(path_to_experiments)
experiments

['IEEE16meshed_con']

In [ ]:
restart_each = 8 # this paramters forces the notebooks to be restarted once in a while, to avoid memory issues
runs = 0

for experiment_folder in [experiment_folder,]:
    path_to_experiments = path_to_project + '/experiments/%s/' % experiment_folder
    experiments = os.listdir(path_to_experiments)
    for experiment_name in experiments:
        print('#' *25, experiment_name, '#' * 25)
        grid_names_list = os.listdir(path_to_experiments + '/%s' % experiment_name)
        for grid_name in grid_names_list:
            print('Grid name:', grid_name)
            scenarios, scenario_generator = load_scenarios(experiment_name, grid_name, path_to_experiments, ptu_int)
            grid = load_grid(experiment_name, grid_name, path_to_experiments)
            for planner in planners:
                #planner.debugging = True
                #planner.tee = True
                print(planner.name)
                results_dict = load_results_dict(planner.name, experiment_name, grid_name, path_to_experiments)
                free_sc_inds = [ind for ind in range(len(scenarios)) if overwrite_existing or 
                                ind not in results_dict.keys()]
                if shuffle:
                    np.random.shuffle(free_sc_inds)
                print('Free scenarios', free_sc_inds)
                for sc_ind in free_sc_inds[:n_scenarios]:
                    if (runs + 1) % restart_each == 0:
                        print('Restarting kernel in 2 seconds')
                        time.sleep(2)
                        restart_kernel_and_run_all_cells()
                    print('Running scenario', sc_ind)
                    scenario = scenarios[sc_ind]
                    env = GridEnv(grid, scenario, scenario_generator)
                    result = simulate(env, planner, tee=False)
                    runs += 1
                    results_dict[sc_ind] = result
                    save_results_dict(results_dict, planner.name, experiment_name, grid_name, path_to_experiments)
            print()
            gc.collect()

######################### IEEE16meshed_con #########################
Grid name: IEEE16meshed_con_g=15_i-line-max=10_p-gen-min=-14444
SOCPPlanner_normalize=True_future=known-future_ev-locations=full_None
Loaded results dict
Free scenarios []
SOCPPlanner_normalize=True_future=known-future_ev-locations=blind_None
Loaded results dict
Free scenarios []
SOCPPlanner_normalize=True_future=known-future_ev-locations=full_single-node
Loaded results dict
Free scenarios []
SOCPPlanner_normalize=True_future=known-future_ev-locations=full_parallel
Loaded results dict
Free scenarios [3, 4, 5]
Running scenario 3
Running scenario 4


In [ ]:
vcv